1. Load dependencies

In [ ]:
import zarr

2. Configure a data store

In [ ]:
store = zarr.storage.FsspecStore.from_url(
    's3://nasa-power/merra2/spatial/power_merra2_daily_spatial_utc.zarr',
    read_only=True,
    storage_options={'anon': True}
)
group = zarr.open_group(store=store, mode='r')

3. Examine the group

In [ ]:
# examine the group
for name, array in group.arrays():
    print(f"Array name: {name}, info: {array.metadata.attributes['long_name']}")

# Get detailed info about a specific array
temperature_array = group['T2M']
print(f"T2M definition: {temperature_array.metadata.attributes['definition']}")

4. Read locations from file

In [ ]:
import csv

locations = list(csv.DictReader(open('assets/locations.csv')))
print(f'Loaded {len(locations)} locations')
for loc in locations[:3]:
    print(loc)

5. Create a function to extract property values for a particular location and date range

In [ ]:
def extract_property_values(array_name, lat, lon, start_date, end_date):
    import numpy as np
    import pandas as pd

    array = group[array_name]
    latitudes = group['lat'][:]
    longitudes = group['lon'][:]
    times = pd.to_datetime(group['time'][:], unit='D')

    lat_idx = (np.abs(latitudes - lat)).argmin()
    lon_idx = (np.abs(longitudes - lon)).argmin()

    time_mask = (times >= pd.to_datetime(start_date)) & (times <= pd.to_datetime(end_date))
    time_indices = np.where(time_mask)[0]
    selected_times = times[time_mask]

    values = array[time_indices, lat_idx, lon_idx]

    units = array.attrs.get('units', 'unknown')

    return pd.DataFrame({'date': selected_times, array_name: values, 'units': units})

6. Extract average temperatures for each location for one year

In [ ]:
for location in locations:
    lat = float(location['lat'])
    lon = float(location['lon'])
    df = extract_property_values('T2M', lat, lon, '2009-01-01', '2009-12-31')
    avg_temp = df['T2M'].mean()
    print(f"Average temperature for ({location['lat']}, {location['lon']}) in 2009: {avg_temp:.2f} {df['units'].iloc[0]}")